In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
# from transformers import pipeline
# pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '103.82.242.92',
        'port': '5710',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData(bind=engine)

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
#
#
select_query = "select 	a.*, b.*, c.sentiment as sentiment_desc                 \
from 	hashtag_sentiment a                  \
		inner join timemarker b              \
		on a.timemarker_id = b.id            \
		inner join master_sentiment c        \
		on a.sentiment = c.sentiment_value"

res_ds = execute_query(select_query)
print(len(res_ds))

10


In [4]:
res_ds.head()

,id,sentiment,db_id,post_at,user_id,tt_name,tti_flag,tt_name_varchar,timemarker_id,flag_date,id,begin_datetime,sentiment_desc
0,4253,2,1,2021-02-14 12:28:32,1,GAR ITB,1,GAR ITB,25858,2021-02-13,25858,2021-02-13 06:15:02,Politik Negatif
1,4084,1,1,2021-02-05 15:48:17,1,Vaksin Boosting Herd Immunity,1,Vaksin Boosting Herd Immunity,23286,None,23286,2021-01-17 11:15:01,Politik Positif
2,4085,1,1,2021-02-05 15:56:28,1,#DoaUntukNegeriku,1,#DoaUntukNegeriku,23373,None,23373,2021-01-18 09:00:01,Politik Positif
3,4086,3,1,2021-02-05 15:56:40,1,#ikatancintaep127,1,#ikatancintaep127,23421,None,23421,2021-01-18 21:00:01,Umum
4,4087,2,1,2021-02-05 15:56:52,1,#JokowiRajaUtang,1,#JokowiRajaUtang,23385,None,23385,2021-01-18 12:00:01,Politik Negatif


In [5]:
res_ds['begin_datetime'] = pd.to_datetime(res_ds['begin_datetime']).dt.date

for index, row in res_ds.iterrows():
    # print(row['tt_name_varchar'])
    # print(row['begin_datetime'])
    
    #
    # search for ttidata
    #
    strSQL = "select a.id from ttidata a inner join timemarker b \
                on a.timemarker_id = b.id \
                where trim(lower(a.""name"")) = trim(lower('"+ row['tt_name_varchar'] +"')) \
                and date_trunc('day',b.begin_datetime) = '" + str(row['begin_datetime']) + "' limit 1"
    #print(strSQL)
    #execute search
    ds_tti_res = execute_query(strSQL)
    # print(len(ds_tti_res))
    if len(ds_tti_res) > 0:
        # update row
        for ds_tti_idx, ds_tti_row in ds_tti_res.iterrows():
            strUpdate = "update ttidata set flag_date = '" + str(row['begin_datetime']) +"',sentiment_value = '" + str(row['sentiment']) + "', sentiment_desc = '" + row['sentiment_desc'] + "' where id = '" + str(ds_tti_row['id']) + "'"
            # print(strUpdate)
            execute_query(strUpdate)
    
    #
    # search for ttwdata
    #
    strSQL = "select a.id from ttwdata a inner join timemarker b \
                on a.timemarker_id = b.id \
                where trim(lower(a.""name"")) = trim(lower('"+ row['tt_name_varchar'] +"')) \
                and date_trunc('day',b.begin_datetime) = '" + str(row['begin_datetime']) + "'"
    #print(strSQL)
    #execute search
    ds_ttw_res = execute_query(strSQL)
    if len(ds_ttw_res) > 0:
        # update row
        for ds_ttw_idx, ds_ttw_row in ds_ttw_res.iterrows():
            strUpdate = "update ttwdata set flag_date = '" + str(row['begin_datetime']) +"',sentiment_value = '" + str(row['sentiment']) + "', sentiment_desc = '" + row['sentiment_desc'] + "' where id = '" + str(ds_ttw_row['id']) + "'"
            # print(strUpdate)
            execute_query(strUpdate)

    #
    # search for ret_available_db
    #
    strSQL = "select a.id from ret_available_db a \
            where trim(lower(a.keyword)) = trim(lower('" + row['tt_name_varchar'] + "')) \
            and date_trunc('day',a.since) = '"+ str(row['begin_datetime']) +"'"
    # print(row['tt_name_varchar'])
    # execute search
    # print(strSQL)
    ds_rad_res = execute_query(strSQL)    
    # print(len(ds_rad_res))
    
    # update rad
    if len(ds_rad_res) > 0:
        for rad_row_idx, rad_row in ds_rad_res.iterrows():
        # for index, row in res_ds.iterrows():
            # print(rad_row['id'])
            strUpdate = "update ret_available_db set flag_date = '" + str(row['begin_datetime']) + "', sentiment_value = '" + str(row['sentiment']) + "', sentiment_desc = '"+ row['sentiment_desc'] +"' where id = '" + str(rad_row['id']) + "'"
            # print(strUpdate)
            execute_query(strUpdate)
            

Query executed successfully. 1 rows affected.
GAR ITB
Query executed successfully. 1 rows affected.
Vaksin Boosting Herd Immunity
Query executed successfully. 1 rows affected.
#DoaUntukNegeriku
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
#ikatancintaep127
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
#JokowiRajaUtang
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
#KunciKebangkitanNKRI
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
#MunarmanNgumpet
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
#PejuangIslamYangMulia
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
#PrayforManado
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
#PrayforManado
Query executed successfully. 1 rows

/home/haviz/cekmedsos/bin/jupyter-nbconvert --ExecutePreprocessor.timeout=21600 --to notebook --execute /home/haviz/multipool/ai-case-study/update-sentiment-table.ipynb